In [46]:
import pandas as pd
import numpy as np
import json
import itertools
import scipy.stats as stats

%matplotlib inline

In [90]:
before_normalization = pd.read_csv('../responses_data/responses.csv')

/usr/local/anaconda3/envs/asp/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (4,5,51) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [91]:
# pd.concat([before_normalization['Q1_entropy'], before_normalization['Q2_entropy']]).describe()
# pd.concat([before_normalization['Q1_entropy'], before_normalization['Q2_entropy']]).plot.hist()

In [92]:
# normalize the data by ensuring that entropy q1 <= entropy q2.

to_swap = before_normalization[before_normalization['Q1_entropy'] > before_normalization['Q2_entropy']]
no_swap = before_normalization[before_normalization['Q1_entropy'] <= before_normalization['Q2_entropy']]

swapped = to_swap.rename(index=str, columns={
    'Q1_entropy': 'Q2_entropy',
    'Q2_entropy': 'Q1_entropy',
    'Q1_channel': 'Q2_channel',
    'Q2_channel': 'Q1_channel',
    'Q1_entropy_class': 'Q2_entropy_class',
    'Q2_entropy_class': 'Q1_entropy_class'
})

normalized = pd.concat([no_swap, swapped])

In [93]:
# limit to one task

normalized = normalized[normalized['taskGroup'] == 'readValue']
len(normalized)

46080

In [94]:
# put entropy into quantiles (low, high)

normalized['Q1_entropy_group'] = pd.qcut(normalized['Q1_entropy'], 2, labels=["L", "H"])
normalized['Q2_entropy_group'] = pd.qcut(normalized['Q2_entropy'], 2, labels=["L", 'H'])

# entropy_cut = 0.3
# normalized.loc[normalized['Q1_entropy'] > entropy_cut, 'Q1_entropy_group'] = 'H'
# normalized.loc[normalized['Q1_entropy'] <= entropy_cut, 'Q1_entropy_group'] = 'L'
# normalized.loc[normalized['Q2_entropy'] > entropy_cut, 'Q2_entropy_group'] = 'H'
# normalized.loc[normalized['Q2_entropy'] <= entropy_cut, 'Q2_entropy_group'] = 'L'

In [95]:
gb = normalized.groupby([
    'taskGroup', 'cardinality', 'nPerCategory',
    'Q1_channel', 'Q2_channel', 'name_channel',
    'Q1_entropy_class', 'Q2_entropy_class',
#    'Q1_entropy_group', 'Q2_entropy_group'
])

In [96]:
row = normalized.iloc[0]
# row

In [97]:
def parse(group):
    cardinality = int(group['cardinality'].iloc[0])
    num_rows = cardinality * int(group['nPerCategory'].iloc[0])

    q1_entropy = group['Q1_entropy'].mean();
    q2_entropy = group['Q2_entropy'].mean();
    
    Q1_entropy_class = group['Q1_entropy_class'].iloc[0];
    Q2_entropy_class = group['Q2_entropy_class'].iloc[0];
    
    # we can identify the same dataset with the key
    data_key = f'{num_rows},{Q1_entropy_class},{Q2_entropy_class}'
    
    fields = [
        {'name': 'n', 'type': 'string', 'entropy': 1, 'cardinality': cardinality},
        {'name': 'q1', 'type': 'number', 'entropy': f'{q1_entropy:.3}', 'cardinality': num_rows},
        {'name': 'q2', 'type': 'number', 'entropy': f'{q2_entropy:.3}', 'cardinality': num_rows}
    ]

    enc = {}

    enc[group['Q1_channel'].iloc[0]] = {'field': 'q1', 'type': 'quantitative'}
    enc[group['Q2_channel'].iloc[0]] = {'field': 'q2', 'type': 'quantitative'}
    enc[group['name_channel'].iloc[0]] = {'field': 'n', 'type': 'nominal'}

    spec = {
        'mark': 'point',
        'encoding': enc
    }

    return {
        'data_key': data_key,
        'fields': fields,
        'num_rows': num_rows,
        'task': group['taskGroup'].iloc[0],
        'spec': spec,
        'score1': group['isCorrect'].mean(),
        'score2': group['completionTime'].mean(),
        'isCorrect': group['isCorrect'],
        'completionTime': group['completionTime'],
    }

In [98]:
specs = pd.DataFrame(columns=['data_key', 'fields', 'num_rows', 'task', 'spec',
                              'score1', 'score2', 'isCorrect', 'completionTime'])
for name, group in gb:
    parsed = parse(group)
    specs = specs.append(parsed, ignore_index=True)
    
print('Number of groups', len(specs))

Number of groups 216


In [99]:
specs = specs.sort_values(['score1', 'score2'])  # sort by correct ratio, then completion time
grouped_specs = specs.groupby(['data_key', 'num_rows', 'task'])

print(len(grouped_specs))

18


In [100]:
grouped_specs.describe()
# specs[:10]

score1                                          \
                             count      mean       std       min       25%   
data_key num_rows task                                                       
30,H,H   30       readValue   12.0  0.928784  0.041968  0.855556  0.903125   
30,L,H   30       readValue   12.0  0.918490  0.037629  0.853125  0.891406   
30,L,L   30       readValue   12.0  0.921974  0.047350  0.805000  0.916071   
300,H,H  300      readValue   12.0  0.906556  0.062247  0.800000  0.863839   
300,L,H  300      readValue   12.0  0.901563  0.056005  0.834375  0.859375   
300,L,L  300      readValue   12.0  0.850278  0.084796  0.678571  0.811111   
60,H,H   60       readValue   12.0  0.928236  0.030416  0.870000  0.910714   
60,L,H   60       readValue   12.0  0.918229  0.031808  0.846875  0.905469   
60,L,L   60       readValue   12.0  0.893463  0.062416  0.750000  0.871111   
600,H,H  600      readValue   12.0  0.873862  0.062076  0.766667  0.832500   
600,L,H  600      readValue   12.0  0.891927  0.058984  0.818750  0.855469   
600,L,L  600      readValue   12.0  0.853045  0.139356  0.537500  0.848864   
9,H,H    9        readValue   12.0  0.950729  0.035816  0.866667  0.930000   
9,L,H    9        readValue   12.0  0.858854  0.035800  0.790625  0.840625   
9,L,L    9        readValue   12.0  0.909180  0.062866  0.795000  0.885139   
90,H,H   90       readValue   12.0  0.907447  0.085603  0.800000  0.831607   
90,L,H   90       readValue   12.0  0.910677  0.054363  0.828125  0.878125   
90,L,L   90       readValue   12.0  0.858532  0.088009  0.693750  0.814683   

                                                          score2  \
                                  50%       75%       max  count   
data_key num_rows task                                             
30,H,H   30       readValue  0.927679  0.963056  0.987500   12.0   
30,L,H   30       readValue  0.923438  0.948438  0.968750   12.0   
30,L,L   30       readValue  0.930952  0.943182  0.983333   12.0   
300,H,H  300      readValue  0.926190  0.944196  0.988889   12.0   
300,L,H  300      readValue  0.884375  0.965625  0.978125   12.0   
300,L,L  300      readValue  0.885714  0.919062  0.931250   12.0   
60,H,H   60       readValue  0.936607  0.946032  0.966667   12.0   
60,L,H   60       readValue  0.915625  0.947656  0.950000   12.0   
60,L,L   60       readValue  0.912500  0.921429  0.975000   12.0   
600,H,H  600      readValue  0.886429  0.908929  0.966667   12.0   
600,L,H  600      readValue  0.864062  0.947656  0.984375   12.0   
600,L,L  600      readValue  0.907143  0.934375  0.955556   12.0   
9,H,H    9        readValue  0.963571  0.975893  0.988889   12.0   
9,L,H    9        readValue  0.865625  0.875000  0.912500   12.0   
9,L,L    9        readValue  0.927652  0.956667  0.975000   12.0   
90,H,H   90       readValue  0.902778  0.993750  1.000000   12.0   
90,L,H   90       readValue  0.915625  0.957031  0.981250   12.0   
90,L,L   90       readValue  0.851042  0.929464  1.000000   12.0   

                                                                      \
                                     mean           std          min   
data_key num_rows task                                                 
30,H,H   30       readValue   6427.401708   1757.217057  4311.893750   
30,L,H   30       readValue   7060.977344   2052.570214  4660.071875   
30,L,L   30       readValue   5767.099122   1392.007723  4331.521429   
300,H,H  300      readValue   6951.712508   1513.824603  5308.905556   
300,L,H  300      readValue   5737.024479   1446.924363  4096.728125   
300,L,L  300      readValue   6016.408635   1395.929670  3745.437500   
60,H,H   60       readValue   5928.123313   1461.853156  4335.375000   
60,L,H   60       readValue   7148.717708   2805.228098  4431.293750   
60,L,L   60       readValue   6222.151474   1548.047912  4611.905556   
600,H,H  600      readValue   7409.591190   2910.292566  5063.575000   
600,L,H  600      readVa

In [101]:
training = []

for name, group in grouped_specs:
    # for neg, pos in zip(range(len(group) - 1), range(1, len(group))):
    for neg, pos in itertools.combinations(range(len(group)), 2):
        negative = group.iloc[neg]
        positive = group.iloc[pos]
        
        # run Welch's t-test
        t, p = stats.ttest_ind(negative.isCorrect, positive.isCorrect, equal_var=False)
#         if p > .05:
#             t, p = stats.ttest_ind(first.completionTime, second.completionTime, equal_var=False)
        if p <= .01:
            training.append({
                'fields': group.iloc[0]['fields'],
                'num_rows': group.iloc[0]['num_rows'],
                'task': group.iloc[0]['task'],
                'negative': negative.spec,
                'positive': positive.spec,
                'p-value': p
            })
    
#     g = list(group.iterrows())
#     previous = g[0][1]
#     for idx, row in g[1:]:
#         spec = row.spec
#         training.append({
#             'fields': row['fields'],
#             'num_rows': row['num_rows'],
#             'task': row['task'],
#             'worse': previous.spec,
#             'better': spec
#         })
#         previous = row

In [103]:
len(training)

400

In [104]:
training[0]

{'fields': [{'cardinality': 10, 'entropy': 1, 'name': 'n', 'type': 'string'},
  {'cardinality': 30, 'entropy': '0.878', 'name': 'q1', 'type': 'number'},
  {'cardinality': 30, 'entropy': '0.91', 'name': 'q2', 'type': 'number'}],
 'negative': {'encoding': {'size': {'field': 'q2', 'type': 'quantitative'},
   'x': {'field': 'q1', 'type': 'quantitative'},
   'y': {'field': 'n', 'type': 'nominal'}},
  'mark': 'point'},
 'num_rows': 30,
 'p-value': 0.008244629195908237,
 'positive': {'encoding': {'row': {'field': 'n', 'type': 'nominal'},
   'x': {'field': 'q2', 'type': 'quantitative'},
   'y': {'field': 'q1', 'type': 'quantitative'}},
  'mark': 'point'},
 'task': 'readValue'}

In [105]:
with open('../data/training/q_q_n.json', 'w') as f:
    json.dump(training, f, indent=2)